# S2W6D7:本周复习

而且，这次我们不依赖 Hugging Face 的高级封装，我们要写一个\*\*“白盒”脚本\*\*，这才是算法工程师的底气。

### 🚀 1 微型推理脚本 (The Micro-Pipeline)

我们要写一段代码，模拟**模型上线部署**后的样子。
**任务**：输入两句话，输出它们的相似度。要求**全链路本地化**，不联网。

请直接运行以下代码，这是你这一周心血的结晶：

In [2]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel
import os

class SimpleInferencePipeline:
    def __init__(self, model_dir):
        # 1. 加载本地资产 (Day 1 & Day 3 的成果)
        # 模拟生产环境：路径是死的，不能依赖网络
        print(f"📡 正在加载模型资产: {model_dir} ...")
        self.tokenizer = BertTokenizer.from_pretrained(model_dir)
        self.model = BertModel.from_pretrained(model_dir)
        self.model.eval() # 必加！(Day 3 重点)
        print("✅ 模型加载完毕！")

    def get_embedding(self, text):
        # 2. 预处理 (Day 1 & Day 2 的成果)
        # 生产环境通常一条条处理，不需要 Collate_fn
        inputs = self.tokenizer(
            text, 
            return_tensors="pt", 
            padding=True,      # 单条其实Pad不Pad无所谓，但为了代码通用性保留
            truncation=True, 
            max_length=512
        )
        
        # 3. 模型推理 (Day 3 的成果)
        with torch.no_grad(): # 必加！省显存
            outputs = self.model(**inputs)
            
        # 4. 向量获取 (Day 3 的避坑指南)
        # 避坑：原生 BERT pooler_output 效果不好，这里我们用 Mean Pooling
        # last_hidden_state: [1, Seq_Len, 768]
        token_embeddings = outputs.last_hidden_state
        attention_mask = inputs['attention_mask']
        
        # 手写 Mean Pooling (排除 Padding 的影响)
        # 这是一个进阶技巧：只对 mask=1 的部分求平均
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    def compute_similarity(self, text1, text2):
        # 5. 相似度计算
        vec1 = self.get_embedding(text1)
        vec2 = self.get_embedding(text2)
        score = F.cosine_similarity(vec1, vec2)
        return score.item()

# --- 🚀 启动引擎 ---
# 指向你的本地路径
model_path = "../../data/pretrained_models/bert-base-chinese"
pipeline = SimpleInferencePipeline(model_path)

# 测试
t1 = "请把桌子上的苹果拿给我"
t2 = "抓取位于桌面的红色水果"
t3 = "向左旋转 90 度"

score_pos = pipeline.compute_similarity(t1, t2)
score_neg = pipeline.compute_similarity(t1, t3)

print(f"\n匹配结果:")
print(f"'{t1}' vs '{t2}': {score_pos:.4f}")
print(f"'{t1}' vs '{t3}': {score_neg:.4f}")

📡 正在加载模型资产: ../../data/pretrained_models/bert-base-chinese ...
✅ 模型加载完毕！

匹配结果:
'请把桌子上的苹果拿给我' vs '抓取位于桌面的红色水果': 0.7387
'请把桌子上的苹果拿给我' vs '向左旋转 90 度': 0.6463


**观察重点**：
这段代码里，我们特意手写了一个 `Mean Pooling` 的逻辑（而不是简单的 `.mean()`），这考虑了 Mask 的影响，比 Hugging Face 默认的更严谨。

# 🎤 2 模拟面试 (Mock Interview Self-Check)

这些问题是你下周面试可能会遇到的“必杀技”。我们来深度拆解。

## 🔥 Q1: `bert-base-chinese` 的词表大小是多少？它是怎么定的？

  * **标准答案**:
    > **21128**。
  * **深度解析 (追问)**:
      * 它是包含了常见的**汉字**（约 2 万个）、**希腊字母**、**俄语字母**、**图形符号**以及 **[CLS], [SEP], [PAD], [UNK], [MASK]** 等特殊 Token。
      * *面试坑点*：它并不包含所有汉字。生僻字会被映射为 `[UNK]`。
      * 相比之下，英文 `bert-base-uncased` 是 **30522**。

## 🔥 Q2: 为什么 BERT 的输入长度限制通常是 512？

这个问题的含金量极高，涵盖了原理和工程两个维度。

  * **维度 1：数学复杂度 (O(L²))**:

    > Self-Attention 需要计算每个词与其他所有词的相关性，生成一个 $L \times L$ 的 Attention Map。

    >   * $L=512$ 时，矩阵大小是 $26万$。
    >   * $L=1024$ 时，矩阵大小是 $100万$ (翻了4倍)。
    >   * $L=2048$ 时，矩阵大小是 $400万$。
    >     **显存占用是序列长度的平方级增长**。512 是当时 Google 在性能和算力之间找到的一个平衡点。

  * **维度 2：位置编码 (Positional Embedding)**:

    > BERT 使用的是 **绝对位置编码 (Absolute Positional Embedding)**，而且它是**学习出来的 (Learned)**，不是像正弦函数那样算出来的。
    > Google 预训练时只初始化了 512 个位置向量。如果你输入第 513 个字，模型压根就没有对应的位置向量可用（IndexError）。
    > *解决办法*：要么截断，要么使用支持长文本的模型（如 Longformer, RoFormer）。

## 🔥 Q3: `DataCollatorWithPadding` 和你自己写的 `collate_fn` 有什么区别？

这是考察你 Day 4 和 Day 6 的实战经验。

  * **DataCollatorWithPadding (官方轮子)**:
      * **优点**: 极致优化（Rust 实现），速度极快；自动处理 Hugging Face 标准格式。
      * **缺点**: “黑盒”。它默认只处理 `input_ids`, `token_type_ids`, `attention_mask`, `labels` 这几个 Key。如果你有一列叫 `my_custom_feature`，它可能会给你丢掉或者报错。
  * **手写 collate\_fn (Day 6 成果)**:
      * **优点**: **完全可控**。
          * 你可以决定 Padding 用 `0` 还是 `-100`。
          * 你可以决定是否要把 `text` 原始内容也保留在 Batch 里。
          * 你可以做**Data Augmentation (数据增强)**：在打包成 Batch 之前，随机把某些字 mask 掉（BERT 预训练逻辑）。
      * **缺点**: 写起来麻烦，如果 Python 循环写得不好，会拖慢 GPU 等待时间。

### 📝 Week 6 结项清单

恭喜黄组长！你已经完成了从“手写原理”到“工业实战”的蜕变。

1.  ✅ **原理**: 彻底理解了 Transformer 内部的维度变化 `[Batch, Seq, 768]`。
2.  ✅ **工程**: 掌握了 Tokenizer, Model, Datasets, DataLoader 的全套流水线。
3.  ✅ **避坑**: 知道了 BERT 的坍缩问题、静态 Padding 的浪费问题、Hugging Face 的安全加载问题。

**🔜 下周预告 (Stage 2 Week 7): 最终决战 —— 微调 (Fine-tuning)**

下周，我们将不再满足于使用“预训练”的通用 BERT。

  * 我们要训练一个 **懂具身指令** 的专属 BERT。
  * 我们要使用 **Hugging Face Trainer API**。
  * 我们要看着 **Loss 曲线** 下降，亲手打造你的第一个 NLP 落地项目。